# Proyek Klasifikasi Gambar: 10 Jenis Hewan (Dataset Animals-10)

- Nama: Bimoseno Kuma
- Email: kuma24@student.ub.ac.id
- ID Dicoding: kukuma

## Import Semua Packages/Library yang Digunakan

In [ ]:
# Install splitfolders
!pip install split-folders --quiet

# Install kaggle
!pip install kaggle --quiet

In [ ]:
import os
import shutil
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import splitfolders

### Data Loading

In [ ]:
# Buat direktori .kaggle jika belum ada dan salin file API key
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Unduh dan unzip dataset Animals-10
!kaggle datasets download -d alessiocorrado99/animals10 -p . --unzip
print("Dataset berhasil diunduh dan diekstrak.")

Dataset URL: https://www.kaggle.com/datasets/alessiocorrado99/animals10
License(s): GPL-2.0
 96% 561M/586M [00:10<00:00, 35.3MB/s]
100% 586M/586M [00:10<00:00, 56.2MB/s]
Dataset berhasil diunduh dan diekstrak.


### Data Preprocessing

#### Split Dataset

In [ ]:
# Path input dan output
input_folder = 'raw-img'
output_folder = 'dataset_split'

# Hapus folder output jika sudah ada untuk memastikan kebersihan
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)

# Lakukan pembagian dengan rasio 80:10:10
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(.8, .1, .1))

print(f"Dataset berhasil dibagi ke dalam folder: {output_folder}")
print(f"Isi folder output: {os.listdir(output_folder)}")

Copying files: 26179 files [00:08, 3185.59 files/s]

Dataset berhasil dibagi ke dalam folder: dataset_split
Isi folder output: ['val', 'test', 'train']


In [ ]:
# Tentukan path direktori yang sudah dibagi di Colab
train_dir = os.path.join(output_folder, 'train')
val_dir = os.path.join(output_folder, 'val')
test_dir = os.path.join(output_folder, 'test')

# Ukuran gambar yang akan digunakan untuk model
IMG_WIDTH, IMG_HEIGHT = 150, 150
BATCH_SIZE = 32

# Generator untuk data latih dengan augmentasi
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generator untuk data validasi dan tes (hanya normalisasi)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Buat flow dari direktori
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

validation_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Dapatkan nama kelas
class_names = list(train_generator.class_indices.keys())
print(f"Nama kelas yang ditemukan: {class_names}")

Found 20938 images belonging to 10 classes.
Found 2614 images belonging to 10 classes.
Found 2627 images belonging to 10 classes.
Nama kelas yang ditemukan: ['cane', 'cavallo', 'elefante', 'farfalla', 'gallina', 'gatto', 'mucca', 'pecora', 'ragno', 'scoiattolo']


## Modelling

In [ ]:
# Membangun arsitektur model
model = Sequential([
    # Blok Konvolusi 1
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Blok Konvolusi 2
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Blok Konvolusi 3
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Blok Konvolusi 4
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Flatten layer untuk mengubah menjadi 1D
    Flatten(),

    # Lapisan Dense
    Dense(512, activation='relu'),
    Dropout(0.5),

    # Lapisan Output
    Dense(len(class_names), activation='softmax') # Jumlah neuron sesuai jumlah kelas
])

# Kompilasi model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Tampilkan ringkasan model
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 148, 148, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 72, 72, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 34, 34, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 15, 15, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     6,423,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,818,506 (26.01 MB)

 Trainable params: 6,817,546 (26.01 MB)

 Non-trainable params: 960 (3.75 KB)

## Evaluasi dan Visualisasi

In [ ]:
# Definisikan callbacks
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        filepath='best_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )
]

In [ ]:
# Tentukan jumlah epoch
EPOCHS = 50

# Latih model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
107/654 ━━━━━━━━━━━━━━━━━━━━ 25:05 3s/step - accuracy: 0.1665 - loss: 7.4022

KeyboardInterrupt: 

In [ ]:
# Plot akurasi dan loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(14, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.grid(True)
plt.show()

In [ ]:
# Muat model terbaik yang disimpan oleh ModelCheckpoint
model.load_weights('best_model.h5')

# Evaluasi pada data tes
print("Mengevaluasi model pada data tes...")
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print(f'\nAkurasi pada data tes: {test_acc*100:.2f}%')
print(f'Loss pada data tes: {test_loss:.4f}')

# Prediksi untuk classification report dan confusion matrix
Y_pred = model.predict(test_generator, steps=np.ceil(test_generator.samples/BATCH_SIZE))
y_pred_classes = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

# Laporan Klasifikasi
print('\nClassification Report:')
print(classification_report(y_true, y_pred_classes, target_names=class_names))

# Confusion Matrix
conf_mat = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('Kelas Asli')
plt.xlabel('Kelas Prediksi')
plt.show()

## Konversi Model

In [ ]:
# Membuat direktori untuk menyimpan model
os.makedirs('saved_model', exist_ok=True)
os.makedirs('tflite', exist_ok=True)
os.makedirs('tfjs_model', exist_ok=True)

# 1. Simpan ke format SavedModel
model.save('saved_model/animals_model')
print("Model berhasil disimpan dalam format SavedModel.")

# 2. Konversi ke format TF-Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('tflite/animals_model.tflite', 'wb') as f:
    f.write(tflite_model)
print("Model berhasil dikonversi ke format TF-Lite.")

# 3. Konversi ke format TFJS
# Install tensorflowjs
!pip install tensorflowjs --quiet

# Lakukan konversi
!tensorflowjs_converter --input_format=keras_saved_model saved_model/animals_model tfjs_model/
print("Model berhasil dikonversi ke format TFJS.")

## Inference (Optional)

In [ ]:
# Muat kembali model SavedModel (contoh)
loaded_model = tf.keras.models.load_model('saved_model/animals_model')

# Fungsi untuk memuat dan memproses gambar
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(IMG_WIDTH, IMG_HEIGHT))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalisasi
    return img_array

# Pilih kelas acak dan gambar acak dari data tes
random_class = random.choice(class_names)
random_class_path = os.path.join(test_dir, random_class)
random_image_name = random.choice(os.listdir(random_class_path))
test_image_path = os.path.join(random_class_path, random_image_name)

# Proses gambar dan lakukan prediksi
processed_img = load_and_preprocess_image(test_image_path)
prediction = loaded_model.predict(processed_img)
predicted_class_index = np.argmax(prediction)
predicted_class_name = class_names[predicted_class_index]
confidence = np.max(prediction) * 100

# Tampilkan hasil
plt.figure(figsize=(6, 6))
img = Image.open(test_image_path)
plt.title(f"Kelas Asli: {random_class}\nPrediksi: {predicted_class_name}\nKeyakinan: {confidence:.2f}%")
plt.imshow(img)
plt.axis('off')
plt.show()

print("--- BUKTI INFERENSI ---")
print(f"Gambar yang diuji: {test_image_path}")
print(f"Kelas Sebenarnya: {random_class}")
print(f"Hasil Prediksi Model: {predicted_class_name}")
print(f"Tingkat Keyakinan: {confidence:.2f}%")